In [1]:
import sys
from os.path import join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import os

/home/puneeth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/puneeth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/puneeth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/puneeth/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
tf.__version__

'2.0.0-beta1'

In [3]:
train_directory = '/home/puneeth/Desktop/projects/Flower-Classification/data/training/'
validation_directory = '/home/puneeth/Desktop/projects/Flower-Classification/data/validation/'
test_directory = '/home/puneeth/Desktop/projects/Flower-Classification/data/test/'
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)

In [4]:
train_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True,
        rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True,
        vertical_flip=True)
    
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_directory, target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE)

validation_generator = test_datagen.flow_from_directory(validation_directory, target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE)

test_generator = test_datagen.flow_from_directory(test_directory, target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE)

Found 12828 images belonging to 102 classes.
Found 5712 images belonging to 102 classes.
Found 0 images belonging to 0 classes.


In [5]:
vgg = VGG16(weights='imagenet', include_top=True)

vgg.summary()

model = Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

for layer in model.layers[:]:
    layer.trainable = False

model.add(Dense(102, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
history = model.fit_generator(
        generator = train_generator,
        steps_per_epoch=13047/32,
        epochs=100,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=50,
        callbacks=[EarlyStopping(min_delta=0.01)]
)

Epoch 1/100


/home/puneeth/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/puneeth/.local/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


365/407 [=========================>....] - ETA: 18:45 - loss: 1.9141 - accuracy: 0.5769